In [8]:
from lxml import etree
#import numpy as np
import torch
from ultralytics import YOLO

# Загрузка файла анотации в формате "CVAT for images 1.1"
# Из файла берётся массив точек полилинии с разметкой шкалы деления: 0мл, 1мл, ... 20мл
def Calibrate (fAnnotation):
#!!! Дописать проверку наличия файла с аннотацией
    tree = etree.parse(fAnnotation)
    root = tree.getroot()
    result = root.xpath('//polyline[@label="injector meter"]')
#!!! Дописать проверку в конфиге необходимого тега полилинии, и точек внутри
    points = result[0].attrib['points']
    # Сложная конструкция, но по сути разбирает стороку на список списков (двумерный список) из двух значений попутно переводя из строковых значений в числовое значние.
    list_of_points = list(map(lambda x: list(map(float,x.split(','))), points.split(';')))
    # array_of_points = np.array(list_of_points) # Изначально думал работать в NumPy, но предсказания выдают значения в тензорах и потому решил тут тоже использовать тензоры
    array_of_points = torch.tensor(list_of_points)
    #print (array_of_points)
    #return list_of_points
    return array_of_points

def GetEpoxyLevel (model, arrayEpoxyLevel, file):
    # Запускаем предсказание
    results = model(filenameInjectorCam)  # Предсказание по изображению. Возвращается список результатов (т.к. можно передать список кадров или даже видео)
#!!! Дописать проверку на возврат значения неудавшегося предсказания
    # Теоретически может быть список результатов, но берём только одно - первое.
    keypoints = results[0].keypoints  # Keypoints object for pose outputs
    #results[0].show()  # display to screen
    #print ('--- Keypoints: ---\n', keypoints)
    
    # Вычисляем середину диагоналей эллипса. На будущее стоит просто определять bounding box этого эллипса. Модель уже выдает как раз середину.
    # Середина между левой и правой точками большой диагонали эллипса
    big_axis_center = (keypoints.xy[0][2] + keypoints.xy[0][3])/2
    # Середина между дальней и ближней точками малой диагонали эллипса
    small_axis_center = (keypoints.xy[0][4] + keypoints.xy[0][5])/2
    # Середина между серединой большой и серединой малой диагоналей. Иногда может быть не равная серединам каждой из них (при ошибках в определении ключевых точек моделью).
    ellipse_center = (big_axis_center + small_axis_center)/2
    
    # Переносим массив точек шприца на то же устройство рассчета где и тензоры модели предсказаний. Если расчёты велись на CUDA, то лучше там и считать всё остальное.
    arrayEpoxyLevel = arrayEpoxyLevel.to(ellipse_center.device)
    
    # Вычисляем ближайшую калиброванную точку к предсказанной точке (середине эллипса)
    LengthMin = (keypoints.orig_shape[0]**2+keypoints.orig_shape[1]**2)**0.5 # Нужно просто большое значение, но решил указать максимально возможное расстояние на изображении (диагональ)
    LevelMin = 0
    for level, kpt in enumerate(arrayEpoxyLevel):
        Length = torch.norm(ellipse_center - kpt) # Расстояние между предсказанной точкой и калиброванной точкой
        if Length < LengthMin: # Если решим, что хоти чтобы при одинаковом расстоянии показывал значение большего уровня, то тогда поставить знак сравнения <=
            LengthMin = Length # Запоминаем минимальное расстояние
            LevelMin = level # Запоминаем уровень к точке которого расстояние минимальное
        #print ("Уровень: ", level, "\tКалиброванная точка: ", kpt, "\tПредсказанная точка: ", ellipse_center, "\tРасстояние: ", Length)
    #print ("Предсказанный уровень эпоксидки: ", LevelMin)
    return LevelMin
    
# Калибровочный файл в котором хранятся уровни от 0 до 20мл. В виде координат X,Y центра эллипса поверхности эпоксидки на каждом уровне.
filenameAnnotation = '/home/nikolay/opencv/epoxy-supervisor/samples/EpoxyLevelCalibrate.annotations.xml'

# Файл весов обученной для распознавания модели
filenameAIModel = '/home/nikolay/opencv/epoxy-supervisor/weights/epoxy-supervisor.20241228.best.pt'

# Изображение для предсказания уровня эпоксидки
filenameInjectorCam = '/home/nikolay/opencv/epoxy-supervisor/datasets/epoxy-level-1774/images/test/000019.png'
#filenameInjectorCam = [ '/home/nikolay/opencv/epoxy-supervisor/datasets/epoxy-level-2640/images/test/00300010.png',
#                        '/home/nikolay/opencv/epoxy-supervisor/datasets/epoxy-level-2640/images/test/00302625.png']

# Калибруем шприц
arrayEpoxyLevel = Calibrate(filenameAnnotation)

# Загружаем модель
model = YOLO(filenameAIModel)  # load a pretrained model

# Запрос уровня по изображению
GetEpoxyLevel(model, arrayEpoxyLevel, filenameInjectorCam)


image 1/1 /home/nikolay/opencv/epoxy-supervisor/datasets/epoxy-level-1774/images/test/000019.png: 480x640 1 epoxy level in injector, 20.7ms
Speed: 14.0ms preprocess, 20.7ms inference, 57.7ms postprocess per image at shape (1, 3, 480, 640)


10